In [25]:
import sys
sys.path.append('..')

import numpy as np
import cirq
from cirq.contrib.qasm_import import circuit_from_qasm

import daqec

**construct the ideal circuit**

In [26]:
qubits = cirq.LineQubit.range(2)
circ = cirq.Circuit([
    cirq.H(qubits[0]),
    cirq.H(qubits[1]),
    cirq.CNOT(qubits[0], qubits[1]),
    cirq.X.on(qubits[0]),
    cirq.T.on(qubits[1]),
    cirq.CNOT(qubits[1], qubits[0]),
    cirq.ZPowGate(exponent=0.1).on(qubits[0]),
    cirq.S.on(qubits[1]),
    cirq.CNOT(qubits[0], qubits[1]),
    cirq.XPowGate(exponent=0.1).on(qubits[0]),
    cirq.H.on(qubits[1]),
    cirq.CNOT(qubits[1], qubits[0]),
])
circ

0: ───H───@───X───X───Z^0.1───@───X^0.1───X───
          │       │           │           │
1: ───H───X───T───@───S───────X───H───────@───

In [27]:
init_state = np.array([
    [1, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0],
]).astype(complex)
final_state = circ.unitary() @ init_state @ circ.unitary().conj().T
prob_11 = final_state[1, 1].real
prob_11

0.46753024329587667

In [28]:
n = len(qubits)
p_cirq = 0.01
p = 4**n / (4**n - 1) * p_cirq
print(f'p_cirq = {p_cirq}, p = {p}')

p_cirq = 0.01, p = 0.010666666666666666


In [29]:
circ_noise_interleaved = cirq.Circuit()
for moment in circ:
    circ_noise_interleaved.append(moment)
    circ_noise_interleaved.append(cirq.depolarize(p_cirq, 2).on(*qubits))
circ_noise_interleaved

0: ───H───D(0.01)───@───D(0.01)───X───D(0.01)───X───D(0.01)───Z^0.1───D(0.01)───@───D(0.01)───X^0.1───D(0.01)───X───D(0.01)───
          │         │   │             │         │   │                 │         │   │                 │         │   │
1: ───H───#2────────X───#2────────T───#2────────@───#2────────S───────#2────────X───#2────────H───────#2────────@───#2────────

In [30]:
r = len(circ)
p_r = 1 - (1 - p)**r
p_r_cirq = p_r * (4**n - 1) / 4**n
print(f'p_r_cirq = {p_r_cirq}, p = {p_r}')
circ_noise = circ.copy()
circ_noise.append(cirq.depolarize(p_r_cirq, 2).on(*qubits))
circ_noise

p_r_cirq = 0.0770762065590169, p = 0.08221462032961802


0: ───H───@───X───X───Z^0.1───@───X^0.1───X───D(0.0771)───
          │       │           │           │   │
1: ───H───X───T───@───S───────X───H───────@───#2──────────

**noisy simulation comparison**

In [31]:
sim = cirq.DensityMatrixSimulator()

rho1 = sim.simulate(circ_noise_interleaved).final_density_matrix
rho2 = sim.simulate(circ_noise).final_density_matrix

assert np.allclose(rho1, rho2, atol=1e-5), "rho1 and rho2 should be equal!"

**append inverse channel to `circ_noise`**

In [32]:
circ_noise

0: ───H───@───X───X───Z^0.1───@───X^0.1───X───D(0.0771)───
          │       │           │           │   │
1: ───H───X───T───@───S───────X───H───────@───#2──────────

In [33]:
final_state_noise = cirq.DensityMatrixSimulator().simulate(circ_noise).final_density_matrix
prob_11_noise = final_state_noise[1, 1].real
print('disparity = ', prob_11_noise - prob_11)

disparity =  -0.017884277004449056
